In [3]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model, Dataset
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np



In [24]:
dm = DataModule(
    name=["RNAstralign", "archiveII", "ribo500-blast", 'bpRNA'],
    data_type=["dms", 'shape', 'structure'],
    train_split=None,
    force_download=False,
    batch_size=64,
    num_workers=0,
    valid_split=2048,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
    use_error=False,
    max_len=177,
    structure_padding_value=0,
)
dm.setup('fit')

Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
bpRNA: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

bpRNA: Download complete. File saved at data/bpRNA/data.json
Loading dataset from HF
Done!                            


In [19]:
for batch in dm.train_dataloader():
    print(batch.get('dms'))
    break

tensor([[-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        ...,
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.],
        [-1000., -1000., -1000.,  ..., -1000., -1000., -1000.]],
       device='mps:0')


In [20]:
import json

name = 'yack'
names = ["RNAstralign", "archiveII", "ribo500-blast", 'bpRNA']
data = {**json.load(open(f'./data/{name}/data.json')) for name in names}

In [25]:
from dmsensei.core.util import _pad, base_pairs_to_pairing_matrix
import tqdm 

def compare_data(dt, data_line, from_ds, ref):
    from_ds = from_ds.cpu().numpy().round(4)
    
    if not dt in data_line:
        assert from_ds is None or set(from_ds) == set([-1000.]), f'{dt} is None in the json and not not in the datamodule for {ref}: {from_ds}'
        return
    L = len(data_line['sequence'])
    data_line = np.array(data_line[dt])
    
    if dt == 'structure':
        data_line = base_pairs_to_pairing_matrix(data_line, L, padding=len(from_ds), pad_value=0).numpy().round(4)

    else:
        from_ds = np.array(from_ds)[:L].round(4)
        data_line = data_line.round(4)
    
    if not np.allclose(data_line, from_ds, atol=1e-6):
        assert 0, f'{dt} keys do not match for {ref}: \ndata_line:\t{data_line.tolist()} \nfrom_ds:\t{from_ds.tolist()}'
    return

def sanity_check(data:dict, dm:DataModule):
    for batch in tqdm.tqdm(dm.train_dataloader(), total=len(dm.train_dataloader())):
        for ref, dms, shape, structure in zip(batch.get('reference'), batch.get('dms'), batch.get('shape'), batch.get('structure')):
            for dt, from_ds in zip(['dms', 'shape', 'structure'], [dms, shape, structure]):
                compare_data(dt, data[ref], from_ds, ref)

sanity_check(data, dm)

  0%|          | 0/2006 [00:00<?, ?it/s]


KeyError: 'bpRNA_CRW_20959'